In [ ]:
__autor__='Victor Garcia'

In [ ]:
import sys
#!{sys.executable} -m pip install zipfile36
#!{sys.executable} -m pip install pysrt

In [8]:
import queue
import urllib.request
import re
from urllib.parse import urljoin
import numpy as np
from scipy.stats import mode
from selenium import webdriver
import os
import pysrt
import zipfile
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime

In [ ]:
#Extraccion

def descargar(pagina):
    try:
        peticion = urllib.request.Request(pagina)  
        html = urllib.request.urlopen(peticion).read()
        print("[*] Descarga OK >>", pagina)
    except:
        print('[!] Error descargando',pagina)
        return None
        
    return html
    
def rastrearEnlaces(pagina,c):
    buscaEnlaces = re.compile('<a[^>]+href="({}\-\d+\.html)"'.format(c), re.IGNORECASE)
    cola = queue.Queue()
    cola.put(pagina) 
    visitados = [pagina]
    print("Buscando enlaces en",pagina)
    while (cola.qsize() > 0):
        html = descargar(cola.get())
        if html == None:
            continue
        enlaces = buscaEnlaces.findall(str(html))
        for enlace in enlaces:
            enlace = urljoin(pagina, str(enlace))
            if(enlace not in visitados):
                cola.put(enlace)
                visitados.append(enlace)
    return visitados

def web_scraper(url):
    for i in range(1,13):
        temporada = rastrearEnlaces(url.format(i),'episode')
        temporada = temporada[1:]

        libreto=[]
        for e in temporada:
            print('\n*******episodios*******\n')
            episodio = rastrearEnlaces(e,'/subtitle')

            print('\n******subtitulos********\n')
            subtitulo = rastrearEnlaces(episodio[1],'download')
            libreto.append(subtitulo[1])

        navegador = webdriver.Chrome()
        libreto=set(libreto)
        for i in libreto:
            navegador.get(i)

#Descomprime los archivos
def descomprime():
    lista_subtitulos=os.listdir('C:/Users/A/Downloads')[1:]#('./subtitulos_zip')
    lista_subtitulos

    for path in lista_subtitulos:
        path="C:/Users/A/Downloads/"+path
        zf=zipfile.ZipFile(path, "r")
        for i in zf.namelist():
            zf.extract(i, path='./subtitulos')

In [ ]:
#Transformacion

def clean():
    lista_subtitulos=os.listdir('./subtitulos')

    for path in lista_subtitulos:
        try:
            path="./subtitulos/"+path
            subs = pysrt.open(path)
            archivo = open("chatbot.txt", "a+")

            for i in range(len(subs)):
                k=subs[i].text.replace('<i>', '').replace('</i>', ' ').replace('\n', ' ').replace('  ', ' ').replace('.', '')\
                         .replace('?','').replace('!','').replace('(','').replace(')','').replace('"','').replace('o/~','')
                k+='. '
                try:
                    print(k, file=archivo)
                except:
                    pass
            archivo.close()
        except:
            pass

In [ ]:
url=["http://es.tvsubtitles.net/tvshow-154-{}.html", 'http://es.tvsubtitles.net/tvshow-1810-{}.html']
for u in url:
    web_scraper(u)

In [ ]:
descomprime()

In [ ]:
clean()

In [3]:
#serie=[[1,17],[2,23],[3,23],[4,24],[5,24],[6,24],[7,24],[8,24],[9,24],[10,24]]
url_page = 'https://bigbangtrans.wordpress.com/series-{}-episode-{}'

libreto=[]

for episodio in range(1,18):
    page = requests.get(url_page.format(1, episodio)).text
    soup = BeautifulSoup(page, "html5")
    text = soup.find_all('span', attrs={'style': 'font-size:small;font-family:Calibri;'})
    for element in text:
        libreto.append(element.text)

['Sheldon: So if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits. If it’s unobserved it will, however, if it’s observed after it’s left the plane but before it hits its target, it will not have gone through both slits.',
 'Leonard: Agreed, what’s your point?',
 'Sheldon: There’s no point, I just think it’s a good idea for a tee-shirt. ',
 'Leonard: Excuse me?',
 'Receptionist: Hang on. ',
 'Leonard: One across is Aegean, eight down is Nabakov, twenty-six across is MCM, fourteen down is… move your finger… phylum, which makes fourteen across Port-au-Prince. See, Papa Doc’s capital idea, that’s Port-au-Prince. Haiti. ',
 'Receptionist: Can I help you?',
 'Leonard: Yes. Um, is this the High IQ sperm bank?',
 'Receptionist: If you have to ask, maybe you shouldn’t be here.',
 'Sheldon: I think this is the place.',
 'Receptionist: Fill these out.',
 'Leonard: Thank-you. We’ll be right back.',
 'Receptionist: Oh, take your

In [61]:
def clasifica(x):
    for personaje in ['sheldon','leonard','howard','raj','penny']:
        if personaje in x.lower():
            return personaje
    return False

training_data = []
for i in range(len(libreto)):
    x=libreto[i].split(' ',1)
    if clasifica(x[0]):
        training_data.append({"class":clasifica(x[0]), "sentence":x[1]})
training_data

[{'class': 'sheldon',
  'sentence': 'So if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits. If it’s unobserved it will, however, if it’s observed after it’s left the plane but before it hits its target, it will not have gone through both slits.'},
 {'class': 'leonard', 'sentence': 'Agreed, what’s your point?'},
 {'class': 'sheldon',
  'sentence': 'There’s no point, I just think it’s a good idea for a tee-shirt. '},
 {'class': 'leonard', 'sentence': 'Excuse me?'},
 {'class': 'leonard',
  'sentence': 'One across is Aegean, eight down is Nabakov, twenty-six across is MCM, fourteen down is… move your finger… phylum, which makes fourteen across Port-au-Prince. See, Papa Doc’s capital idea, that’s Port-au-Prince. Haiti. '},
 {'class': 'leonard', 'sentence': 'Yes. Um, is this the High IQ sperm bank?'},
 {'class': 'sheldon', 'sentence': 'I think this is the place.'},
 {'class': 'leonard', 'sentence': 'Thank-you. We’ll be r

In [65]:
import pickle

def guardar_datos(dic):
    with open("libreto.dat", "wb") as f:
        pickle.dump(dic, f)
    f.close ()
guardar_datos(training_data)

In [67]:
with open("libreto.dat", "rb") as f:
    training = pickle.load (f)
training

[{'class': 'sheldon',
  'sentence': 'So if a photon is directed through a plane with two slits in it and either slit is observed it will not go through both slits. If it’s unobserved it will, however, if it’s observed after it’s left the plane but before it hits its target, it will not have gone through both slits.'},
 {'class': 'leonard', 'sentence': 'Agreed, what’s your point?'},
 {'class': 'sheldon',
  'sentence': 'There’s no point, I just think it’s a good idea for a tee-shirt. '},
 {'class': 'leonard', 'sentence': 'Excuse me?'},
 {'class': 'leonard',
  'sentence': 'One across is Aegean, eight down is Nabakov, twenty-six across is MCM, fourteen down is… move your finger… phylum, which makes fourteen across Port-au-Prince. See, Papa Doc’s capital idea, that’s Port-au-Prince. Haiti. '},
 {'class': 'leonard', 'sentence': 'Yes. Um, is this the High IQ sperm bank?'},
 {'class': 'sheldon', 'sentence': 'I think this is the place.'},
 {'class': 'leonard', 'sentence': 'Thank-you. We’ll be r